In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Attention: Motivation

Let's revisit the Encoder-Decoder architecture

The Encoder
- Acts on input sequence $[\x_{(1)} \dots \x_{(\bar{T})}]$
- Producing a sequence of latent states $[ \bar{\h}_{(1)}, \dots, \bar{\h}_{(\bar{T})} ]$



The Decoder
- Acts on the *final* Encoder latent state $\bar{\h}_{(\bar{T})}$
- Producing a sequence of outputs $[ \hat{\y}_{(1)}, \dots, \hat{\y}_{(T)} ]$
- Often feeding step $\tt$ output $\hat{\y}_\tp$ as Encoder input at step $(\tt+1)$



<table>
    <tr>
        <th><center>RNN Encoder/Decoder</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_API_Encoder_Decoder.png"</td>
    </tr>
</table>



The following diagram is a condensed depiction of the process

<table>
    <tr>
        <th><center>Sequence to Sequence: training (teacher forcing) + inference: No attention</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_seq2seq.png"></td>
    </tr>
</table>

Recall that $\bar{\h}_{(\bar{\tt})}$ is a fixed length encoding of the input prefix $\x_{(1)}, \ldots, \x_{(\bar{\tt})} $.

So $\bar{\h}_{(\bar{T})}$, which initializes the Decoder, is a summary of entire input sequence $\x$.

This fact enables us to decouple the Encoder from the Decoder
- The consumption of input $\x$ and production of output $\hat{\y}$ do not have to be synchronized
- Allowing for the possibility that $T \ne \bar{T}$
- For example
    - There is no one to one mapping between languages (nor does ordering of words get preserved)


Let's focus on the part of the Decoder
- That transforms latent state (or short term memory) $\h_\tp$ to output $\hat{\y}_\tp$


<table>
    <tr>
        <th><center>Decoder: No attention</center></th>
    </tr>
    <tr>
        <td><img src="images/Decoder_no_attention.png"></td>
    </tr>
</table>

We can generalize this transformation as
$$\hat{\y}_\tp = D( \h_\tp; \mathbf{s})$$

In the vanilla RNN, this was governed by the equation
$$\hat{\y}_\tp  =   D(\h_\tp; \mathbf{s}) = \W_{hy} \h_\tp  + \b_y$$

Additional parameter $\mathbf{s}$
- Was unused in this example (our illustration used $\bar{\h}_{(\bar{T})}$ as a place-holder)
- But may be used in other cases


This simple mapping of $\h_\tp$ to $\hat{\y}_\tp$ can be extremely burdensome

It is often the case that $\hat{\y}_\tp$
- Depends mostly on a **specific element** $\x_{(\bar{\tt})}$ of the input
- Or on a **specific prefix** of the input: $\x_{(1)}, \ldots, \x_{(\bar{\tt})} $


Consider the example of language translation
- When predicting word $\hat{\y}_\tp$  in the Target language
- Some "context" provided by the Source language may greatly influence the prediction
    - For example: gender/plurality of the subject

This context is usually much smaller
 than the entire sequence $\x$ of length $\bar{T}$.



By not allowing $D(\h_\tp; \mathbf{s})$ *direct* access to the required context, we force the Decoder
- To encode the context of the Source 
- Along with the specific information of the Target
- Into $\h_\tp$

This makes $\h_\tp$ unnecessarily complex and perhaps difficult to learn well.

We will introduce a mechanism called *Attention* to alleviate this burden.

To give you a better feel for context, here are some examples

**Image captioning example**
- Source: Image
- Target: Caption: "A woman is throwing a **frisbee** in a park."
- Attending over *pixels* **not** sequence

<center><strong>Visual attention</strong></center>
<table>
    <tr>
        <td><img src="images/shat_-002-027.jpg"></td>
        <td><img src="images/shat_-002-028.jpg"></td>
    </tr>
    <tr>
        <td colspan=2><center>A woman is throwing a <strong>frisbee</strong> in a park.</center></td>
    </tr>
</table>
Attribution: https://arxiv.org/pdf/1502.03044.pdf


**Image captioning example**
- Source: Image
- Target: Caption: "A giraffe standing in a forest with **trees** in the background."
- Attending over *pixels* **not** sequence

<center><strong>Visual attention</strong></center>
<table>
    <tr>
        <td><img src="images/shat_-002-035.png"></td>
        <td><img src="images/shat_-002-036.jpg"></td>
    </tr>
    <tr>
        <td colspan=2><center>A giraffe standing in a forest with <strong>trees</strong> in the background.</center></td>
    </tr>
</table>
Attribution: https://arxiv.org/pdf/1502.03044.pdf

**Date normalization example**
- Source: Dates in free-form: "Saturday 09 May 2018"
- Target: Dates in normalized form: "2018-05-09"

[link](https://github.com/datalogue/keras-attention#example-visualizations)

# Attend to what's important

The solution to over-loading $\h_\tp$ with Source context is conceptually straight forward.

In the Decoder expression $D(\h_\tp; \mathbf{s})$, let
$$
\mathbf{s} = \c_\tp
$$
where $\c_\tp$ is a variable
- That supplies the appropriate context for output $\hat{\y}_\tp$
- Conditional on $\h_\tp$

Because $\bar{\h}_{(\bar{\tt})}$ 
- Is a fixed length encoding of the input prefix $\x_{(1)}, \ldots, \x_{(\bar{\tt})} $
- It can be assigned to $\c_\tp$ as the context for the prefix of $\x$ of length $\bar{\tt}$

$$ \c_\tp \in \{ \bar{\h}_{(1)}, \dots, \bar{\h}_{(\bar{T})} \} $$


We say
- The Decoder "attends to" (pays attention) $\bar{\h}_{(\bar{\tt})}$
- When generating output $\hat{\y}_\tp$

That is: it focusses its attention on a specific part of the input $\x$

<table>
    <tr>
        <th><center>Decoder: Attention</center></th>
    </tr>
    <tr>
        <td><img src="images/Decoder_attention.png"></td>
    </tr>
</table>

The dotted line from $\h_\tp$ on the left of the Choose box
- Indicates that the Choice is conditional on Decoder state $\h_\tp$

Here is a diagram summarizing the Attention mechanism

<table>
    <tr>
        <th><center>Sequence to Sequence: attention</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_seq2seq_attention.png" width=1000></td>
    </tr>
</table>

How is the choice of $\c_\tp$ from the set $\{ \bar{\h}_{(1)}, \dots, \bar{\h}_{(\bar{T})} \}$ accomplished ?

The "Choose" box
- Is a Neural Network
- With it's own weights
- That learn to make the best choice for the Target task !

In other words
- It is trained as part of the larger task

This is a common technique in Deep Learning that may, at first, appear magical
- Hypothesize the existence of a mechanism to solve your problem
- Train a Neural Network to conjure up the mechanism !

# Just for fun: Attention in action 

Here are some examples of Sequence to Sequence problems using Attention.

**Visual Attention example**
- Source: Image
- Target: Caption: "A giraffe and two zebras standing in a field."
- Attending over *pixels* **not** sequence

<img src="https://raw.githubusercontent.com/yunjey/show_attend_and_tell/master/jpg/train.jpg" width=1000>


Attribution: https://arxiv.org/abs/1502.03044

**Language Translation example**
- Source: Spanish
- Target: English
- Colab notebook !
[Translation example](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb#scrollTo=CiwtNgENbx2g)

# Conclusion

We recognized that the Decoder function responsible for generating Decoder output $\hat{\y}_\tp$
$$
\hat{\y}_\tp = D( \h_\tp; \mathbf{s})
$$

was quite rigid when it ignored argument $\mathbf{s}$.

This rigidity forced Decoder latent state $\h_\tp$ to assume the additional responsibility of including Encoder context.

Attention was presented as a way to obtain Encoder context through argument $\mathbf{s}$.

In [2]:
print("Done")

Done
